# Finance Notebooks #
## Prezzi e rendimenti ##

Prezzi e rendimenti costituiscono la base di qualunque analisi di serie finanziarie! Utilizziamo **python** (https://www.python.org/) come linguaggio di programmazione per svolgere un'analisi introduttiva a questi concetti, avvalendoci di **pandas** (http://pandas.pydata.org/) come strumento di raccolta e manipolazione dei dati, **bqplot** (https://bqplot.readthedocs.io/en/latest/index.html) per la visualizzazione e **Jupyter Widgets** (https://ipywidgets.readthedocs.io/en/stable/) per l'interazione. Prima di tutto, importiamo un po' di moduli per iniziare la nostra analisi.

In [1]:
import pandas as pd
import ipywidgets as widgets
import bqplot as bq

A questo punto, scegliamo qualche azione da analizzare. Prendiamo: **Apple** (AAPL), **General Electric** (GE), **Microsoft** (MSFT), **McDonald's** (MCD), **Coca-Cola** (KO).

In [18]:
# I tickers delle azioni scelte
STOCKS = ['AAPL', 'GE', 'MSFT', 'MCD', 'KO']

Ora, scarichiamo i prezzi dal sito **Macrotrends**. (Sempre più complicato trovare serie finanziarie *free* su internet, al momento Yahoo!. Google e Morningstar sembrano non disponibili).

In [19]:
# pandas ora legge direttamente i file csv scaricati da internet, fornendo un url
def download_px(tickers):
    """ Download csv files with prices for each ticker and merge the in a pandas DataFrame."""
    l = []
    for t in tickers:
        url = 'http://download.macrotrends.net/assets/php/stock_data_export.php?t={}'.format(t)
        df = pd.read_csv(url, header=0, index_col=0, usecols=[0, 4], skiprows=10, parse_dates=True)
        df.columns = [t]
        l.append(df)
    return(pd.concat(l, axis=1))

px = download_px(STOCKS)

Poi prendiamo un ragionevole sottoinsieme di dati, diciamo più o meno gli ultimi dieci anni, avvalendoci delle ottime funzionalità di manipolazione delle date di **pandas**. Mostriamo i primi...

In [20]:
# Facciamo partire la serie dall'ultimo fine mese di dieci anni da oggi (n=121)
prices = px.loc[(pd.Timestamp.today() - pd.tseries.offsets.MonthEnd(n=121)).strftime('%m-%d-%Y'):pd.Timestamp.today().strftime('%m-%d-%Y'), :]
prices.head()

,AAPL,GE,MSFT,MCD,KO
date,,,,,
2009-03-02,12.5629,7.60,15.79,51.85,19.845
2009-03-03,12.6243,7.01,15.88,52.43,19.415
2009-03-04,13.0243,6.69,16.12,53.15,19.865
2009-03-05,12.6914,6.66,15.27,50.86,18.925
2009-03-06,12.1857,7.06,15.28,52.12,19.550


...e gli ultimi valori del periodo.

In [21]:
prices.tail()

,AAPL,GE,MSFT,MCD,KO
date,,,,,
2019-03-19,186.53,10.19,117.65,183.11,45.56
2019-03-20,188.16,10.22,117.52,184.97,45.53
2019-03-21,195.09,10.27,120.22,186.37,45.51
2019-03-22,191.05,9.98,117.05,186.81,45.93
2019-03-25,188.74,9.88,117.66,185.72,46.03


Creiamo adesso un grafico dei prezzi e rendiamolo un po' interattivo consentendo di selezionare le azioni da mostrare con delle **checkbox**.

In [22]:
# Costruiamo una checkbox per ogni titolo
chks_1 = [widgets.Checkbox(description=d, value=True) for d in prices.columns]

# Costruiamo il grafico
x_sc_1 = bq.DateScale()
y_sc_1 = bq.LinearScale()
lines_1 = bq.Lines(x=prices.index, y=[prices[c] for c in prices.columns[[cb.value for cb in chks_1]]], scales={'x': x_sc_1, 'y': y_sc_1},
                   display_legend=True, colors=[bq.colorschemes.CATEGORY10[prices.columns.get_loc(n)] for n in prices.columns],
                   labels=[c for c in prices.columns[[cb.value for cb in chks_1]]])
ax_x_1 = bq.Axis(scale=x_sc_1, grid_lines='solid', grid_color='#DDDDDD')
ax_y_1 = bq.Axis(scale=y_sc_1, orientation='vertical', grid_lines='solid', grid_color='#DDDDDD')
fig_1 = bq.Figure(marks=[lines_1], axes=[ax_x_1, ax_y_1], title='Andamento dei prezzi', legend_location='top-left', background_style={'fill': 'white'},
                  layout={'width': '100%'})

# Definiamo una funzione che aggiorna i grafici, la legenda e i colori a seconda delle checkbox selezionate
def update_plot_1(change):
    """ Update the chart with boolean value from checkboxes."""
    lines_1.y = [prices[c] for c in prices.columns[[cb.value for cb in chks_1]]]
    lines_1.labels = [c for c in prices.columns[[cb.value for cb in chks_1]]]
    lines_1.colors = [bq.colorschemes.CATEGORY10[prices.columns.get_loc(c)] for c in prices.columns[[cb.value for cb in chks_1]]]
    
# Intercettiamo gli eventi relativi alle checkbox
for cb in chks_1:
    cb.observe(update_plot_1, 'value')

# Definiamo il layout delle widget
box_1 = widgets.HBox(chks_1)
widgets.VBox([box_1, fig_1])

Per poter confrontare tra di loro gli andamenti, è necessario normalizzare i prezzi, ad esempio ribasandoli a 100 in una data uguale per tutti $P_t$ mediante la formula $$\frac{100 * P}{P_t}$$ Costruiamo allora un grafico dei prezzi normalizzati, aggiungendo uno **slider** per poter modificare la data di ribasamento.

In [24]:
# Costruiamo lo slider e colleghiamolo alle date della serie storica
opts_2 = [d.strftime('%d-%m-%Y') for d in prices.index]
slider_2 = widgets.SelectionSlider(options=opts_2, description='Data', layout={'width': '100%'})

# Normalizziamo i prezzi utilizzando la formula sopra citata
nprices = 100 * prices / prices.loc[pd.to_datetime(slider_2.value, dayfirst=True), :]

# Costruiamo il grafico
x_sc_2 = bq.DateScale(date_format = "%y")
y_sc_2 = bq.LinearScale()
x_sec_2 = bq.LinearScale(min=0, max=1)
y_sec_2 = bq.LinearScale(min=0, max=1)
lines_2 = bq.Lines(x=nprices.index, y=[nprices[c] for c in nprices.columns], scales={'x': x_sc_2, 'y': y_sc_2},
                   display_legend=True, labels=[c for c in nprices.columns])
frac = (pd.to_datetime(slider_2.value, dayfirst=True) - nprices.index[0]) / (nprices.index[-1] - nprices.index[0])
vline_2 = bq.Lines(x=[frac, frac], y=[0, 1], scales={'x': x_sec_2, 'y': y_sec_2}, colors=['#AAAAAA'])
ax_x_2 = bq.Axis(scale=x_sc_2, grid_lines='solid', grid_color='#DDDDDD')
ax_x_sec_2 = bq.Axis(scale=x_sec_2, visible=False)
ax_y_2 = bq.Axis(scale=y_sc_2, orientation='vertical', grid_lines='solid', grid_color='#DDDDDD')
ax_y_sec_2 = bq.Axis(scale=y_sec_2, orientation='vertical', visible=False)
fig_2 = bq.Figure(marks=[lines_2, vline_2], axes=[ax_x_2, ax_x_sec_2, ax_y_2, ax_y_sec_2], title='Andamento normalizzato dei prezzi ({} = 100)'.format(slider_2.value),
                  legend_location='top-left', background_style={'fill': 'white'}, layout={'width': '100%'})

# Definiamo una funzione che aggiorna la base di normalizzazione dei prezzi, il grafico e il titolo in base ai valori dello slider
def update_plot_2(change):
    """ Update the chart with value from slider."""
    nprices = 100 * prices / prices.loc[pd.to_datetime(slider_2.value, dayfirst=True), :]
    frac = (pd.to_datetime(slider_2.value, dayfirst=True) - nprices.index[0]) / (nprices.index[-1] - nprices.index[0])
    lines_2.y = [nprices[c] for c in nprices.columns]
    vline_2.x = [frac, frac]
    fig_2.title = 'Andamento normalizzato dei prezzi ({} = 100)'.format(slider_2.value)

# Intercettiamo i valori dello slider
slider_2.observe(update_plot_2, 'value')

# Definiamo il layout delle widget
widgets.VBox([slider_2, fig_2])

Ma qual è il rendimento delle azioni nel periodo considerato? Una prima risposta viene dal **rendimento assoluto**, ossia la percentuale di apprezzamento dei titoli sull'intero periodo, calcolata come $$100 * (\frac{P_T}{P_0} - 1)$$ dove $P_T$ è il prezzo del titolo alla fine del periodo e $P_0$ quello all'inizio del periodo.

In [25]:
absolute_returns = (prices.iloc[-1, :] / prices.iloc[0, :] - 1).to_frame().T
absolute_returns.index = ['Rendimenti assoluti (%)']
absolute_returns.style.format('{:.2%}')

,AAPL,GE,MSFT,MCD,KO
Rendimenti assoluti (%),1402.36%,30.00%,645.16%,258.19%,131.95%


Per poter confrontare i rendimenti con - ad esempio - altri investimenti, può essere comodo trasformare il rendimento assoluto in un dato annualizzato. Per far ciò, utilizziamo la formula del **rendimento composto annuo** $$100 * [(\frac{P_T}{P_0})^{\frac{365}{T}} - 1]$$ dove $P_T$ è il prezzo del titolo alla fine del periodo, $P_0$ quello all'inizio del periodo e $T$ è il numero dei giorni compresi nel periodo.

In [26]:
compound_returns = ((1 + absolute_returns) ** (pd.Timedelta(365, unit='D') / (prices.index[-1] - prices.index[0])) - 1)
compound_returns.index = ['Rendimenti composti annui (%)']
compound_returns.style.format('{:.2%}')

,AAPL,GE,MSFT,MCD,KO
Rendimenti composti annui (%),30.88%,2.64%,22.08%,13.51%,8.72%


Ora costruiamo un grafico dei rendimenti, **assoluti** e **composti annui**, aggiungendo dei **radio button** per poter selezionare il tipo di rendimento da mostrare.

In [27]:
# Concateniamo per comodità i dataframe absolute_returns e compound_returns
df_3 = pd.concat([absolute_returns, compound_returns])

# Costruiamo i radio buttons per selezionare il tipo di rendimento da mostrare
opts_3 = [i for i in df_3.index]
radio_3 = widgets.RadioButtons(options=opts_3, description='Tipo rendimento', style={'description_width': 'initial'}, layout={'width': '100%'})

# Costruiamo un grafico a barre con le etichette delle percentuali
x_sc_3 = bq.OrdinalScale()
y_sc_3 = bq.LinearScale(max=max(1.1 * df_3.loc[radio_3.value, :].max(), 0), min=min(1.1 * df_3.loc[radio_3.value, :].min(), 0))
bars_3 = bq.Bars(x=df_3.columns, y=df_3.loc[radio_3.value, :], scales={'x': x_sc_3, 'y': y_sc_3},
                 colors=[bq.colorschemes.CATEGORY10[df_3.index.get_loc(radio_3.value)]])
labels_3 = bq.Label(x=df_3.columns, y=df_3.loc[radio_3.value, :], text=[f'{v:.2%}' for v in df_3.loc[radio_3.value, :]], scales={'x': x_sc_3, 'y': y_sc_3}, 
                    colors=['#AAAAAA'], align='middle', y_offset=-15)
ax_x_3 = bq.Axis(scale=x_sc_3, grid_lines='solid', grid_color='#DDDDDD')
ax_y_3 = bq.Axis(scale=y_sc_3, orientation='vertical', tick_format='0%', grid_lines='solid', grid_color='#DDDDDD')
fig_3 = bq.Figure(marks=[bars_3, labels_3], axes=[ax_x_3, ax_y_3], title=radio_3.value, background_style={'fill': 'white'}, layout={'width': '100%'})

# Definiamo una funzione per aggiornare il grafico, le etichette e il titolo
def update_plot_3(change):
    """ Update the chart with value from radio buttons."""
    y_sc_3.max = max(1.1 * df_3.loc[radio_3.value, :].max(), 0)
    y_sc_3.min = min(1.1 * df_3.loc[radio_3.value, :].min(), 0)    
    bars_3.y = df_3.loc[radio_3.value, :]
    bars_3.colors = [bq.colorschemes.CATEGORY10[df_3.index.get_loc(radio_3.value)]]
    labels_3.y = df_3.loc[radio_3.value, :]
    labels_3.text = [f'{v:.2%}' for v in df_3.loc[radio_3.value, :]]
    fig_3.title = radio_3.value

# Intercettiamo i valori dei radio buttons
radio_3.observe(update_plot_3, 'value')

# Definiamo il layout delle widget
widgets.VBox([radio_3, fig_3])

Una rappresentazione comune dei risultati di uno strumento finanziario si ottiene attraverso il calcolo di **rendimenti percentuali di periodo**. In questo caso, ad esempio, calcoliamo i rendimenti percentuali per anno. 

In [28]:
# pandas ha una funzione - pct_change() - che calcola le variazioni percentuali secondo la formula dei rendimenti assoluti
yearly_returns = prices.resample('A').last().pct_change().iloc[1:, :]
yearly_returns.style.format('{:.2%}')

,AAPL,GE,MSFT,MCD,KO
date,,,,,
2010-12-31 00:00:00,53.07%,20.89%,-8.43%,22.93%,15.39%
2011-12-31 00:00:00,25.56%,-2.08%,-6.99%,30.71%,6.39%
2012-12-31 00:00:00,31.40%,17.20%,2.89%,-12.08%,3.62%
2013-12-31 00:00:00,5.42%,33.54%,40.06%,10.00%,13.96%
2014-12-31 00:00:00,37.72%,-9.85%,24.16%,-3.43%,2.20%
2015-12-31 00:00:00,-4.64%,23.27%,19.44%,26.08%,1.75%
2016-12-31 00:00:00,10.03%,1.44%,12.00%,3.03%,-3.49%
2017-12-31 00:00:00,46.11%,-44.78%,37.66%,41.41%,10.66%
2018-12-31 00:00:00,-6.79%,-56.62%,18.74%,3.17%,3.20%


Dai rendimenti per anno è possibile calcolare un **rendimento medio annualizzato**, da non confondere con il rendimento composto annuo (infatti, i risultati sono diversi!).

In [29]:
yearly_avgs = yearly_returns.mean().to_frame().T
yearly_avgs.index = ['Rendimento medio annuo (%)']
yearly_avgs.style.format('{:.2%}')

,AAPL,GE,MSFT,MCD,KO
Rendimento medio annuo (%),21.75%,1.35%,15.54%,12.64%,5.09%


Possiamo generare un grafico dei rendimenti percentuali annui e delle relative medie, con dei **toggle button** che permettano di scegliere il titolo da visualizzare.

In [30]:
# Costruiamo dei toggle buttons per selezionare il titolo
opts_4 = [c for c in yearly_returns.columns]
buttons_4 = widgets.ToggleButtons(options=opts_4, description='Titolo', layout={'width': '100%'})

# Costruiamo un grafico a barre con le etichette delle percentuali e una linea orizzontale a rappresentare la media
x_sc_4 = bq.DateScale()
y_sc_4 = bq.LinearScale(max=max(1.1 * yearly_returns.loc[:, buttons_4.value].max(), 0), min=min(1.1 * yearly_returns.loc[:, buttons_4.value].min(), 0))
x_sec_4 = bq.OrdinalScale()
x_ter_4 = bq.LinearScale(min=0, max=1)
bars_4 = bq.Bars(x=yearly_returns.index, y=yearly_returns.loc[:, buttons_4.value], scales={'x': x_sc_4, 'y': y_sc_4},
                 colors=[bq.colorschemes.CATEGORY10[yearly_returns.columns.get_loc(buttons_4.value)]])
labels_4 = bq.Label(x=range(len(yearly_returns.index)), y=yearly_returns.loc[:, buttons_4.value], text=[f'{v:.2%}' for v in yearly_returns.loc[:, buttons_4.value]],
                    scales={'x': x_sec_4, 'y': y_sc_4}, colors=['#AAAAAA'], align='middle', y_offset=-15, default_size=13)
avg_line_4 = bq.Lines(x=[0, 1], y=[yearly_avgs.loc[:, buttons_4.value], yearly_avgs.loc[:, buttons_4.value]], scales={'x': x_ter_4, 'y': y_sc_4}, colors=['#666666'], 
                      line_style='dash_dotted', display_legend=True, labels=[f'Rendimento medio annuo: {v:.2%}' for v in yearly_avgs.loc[:, buttons_4.value]])
ax_x_4 = bq.Axis(scale=x_sc_4, grid_lines='solid', grid_color='#DDDDDD')
ax_x_sec_4 = bq.Axis(scale=x_sec_4, visible=False)
ax_x_ter_4 = bq.Axis(scale=x_ter_4, visible=False)
ax_y_4 = bq.Axis(scale=y_sc_4, orientation='vertical', tick_format='0%', grid_lines='solid', grid_color='#DDDDDD')
fig_4 = bq.Figure(marks=[bars_4, labels_4, avg_line_4], axes=[ax_x_4, ax_x_sec_4, ax_x_ter_4, ax_y_4], title=f'Rendimenti percentuali annui per {buttons_4.value}', 
                  background_style={'fill': 'white'}, legend_location='top-right', layout={'width': '100%'})

# Definiamo una funzione per aggiornare il grafico, le etichette, la media e il titolo
def update_plot_4(change):
    """ Update the chart with value from the toggle buttons."""
    y_sc_4.max = max(1.1 * yearly_returns.loc[:, buttons_4.value].max(), 0)
    y_sc_4.min = min(1.1 * yearly_returns.loc[:, buttons_4.value].min(), 0)   
    bars_4.y = yearly_returns.loc[:, buttons_4.value]
    bars_4.colors = [bq.colorschemes.CATEGORY10[yearly_returns.columns.get_loc(buttons_4.value)]]
    labels_4.y = yearly_returns.loc[:, buttons_4.value]
    labels_4.text = [f'{v:.2%}' for v in yearly_returns.loc[:, buttons_4.value]]
    avg_line_4.y = [yearly_avgs.loc[:, buttons_4.value], yearly_avgs.loc[:, buttons_4.value]]
    avg_line_4.labels = [f'Rendimento medio annuo: {v:.2%}' for v in yearly_avgs.loc[:, buttons_4.value]]
    fig_4.title = f'Rendimenti percentuali annui per {buttons_4.value}'

# Intercettiamo i valori dei toggle buttons
buttons_4.observe(update_plot_4, 'value')

# Definiamo il layout delle widget
widgets.VBox([buttons_4, fig_4])

Oltre ai rendimenti percentuali, in finanza è comune calcolare i **rendimenti logaritmici**, definiti come la differenza dei logaritmi dei prezzi a fine e a inizio periodo. In formula $$ln(P_T) - ln(P_0)$$

In [31]:
# Importiamo numpy per avere la funzione logaritmo, non disponibile in pandas
import numpy as np

yearly_log_returns = prices.resample('A').last().apply(np.log).diff().iloc[1:, :]
yearly_log_returns.style.format('{:.2%}')

,AAPL,GE,MSFT,MCD,KO
date,,,,,
2010-12-31 00:00:00,42.57%,18.97%,-8.81%,20.65%,14.31%
2011-12-31 00:00:00,22.76%,-2.10%,-7.24%,26.78%,6.19%
2012-12-31 00:00:00,27.31%,15.87%,2.85%,-12.87%,3.55%
2013-12-31 00:00:00,5.28%,28.92%,33.69%,9.53%,13.07%
2014-12-31 00:00:00,32.01%,-10.37%,21.64%,-3.49%,2.18%
2015-12-31 00:00:00,-4.75%,20.92%,17.76%,23.18%,1.74%
2016-12-31 00:00:00,9.56%,1.43%,11.34%,2.99%,-3.55%
2017-12-31 00:00:00,37.92%,-59.38%,31.96%,34.65%,10.13%
2018-12-31 00:00:00,-7.03%,-83.51%,17.18%,3.12%,3.15%


Anche in questo caso, possiamo calcolare un rendimento medio (logaritmico) annualizzato, diverso da quello percentuale e da quello composto annuo.

In [32]:
yearly_log_avgs = yearly_log_returns.mean().to_frame().T
yearly_log_avgs.index = ['Log rendimento medio annuo (%)']
yearly_log_avgs.style.format('{:.2%}')

,AAPL,GE,MSFT,MCD,KO
Log rendimento medio annuo (%),18.36%,-4.26%,13.51%,10.90%,4.79%


Un interessante proprietà dei **rendimenti logaritmici** è che sono "simmetrici"", ossia una perdita del 30% seguita da un guadagno del 30% in termini logaritmici ci riporta al livello iniziale. Questo non è vero per i rendimenti percentuali! In termini logaritmici, inoltre, è possibile perdere più del 100%.

Per trasformare i rendimenti logaritmici in rendimenti percentuali vale la formula $$R = e^r - 1$$ dove $R$ sono i rendimenti percentuali e $r$ quelli logaritmici.

In [33]:
((yearly_log_returns).apply(np.exp) - 1).style.format('{:.2%}')

,AAPL,GE,MSFT,MCD,KO
date,,,,,
2010-12-31 00:00:00,53.07%,20.89%,-8.43%,22.93%,15.39%
2011-12-31 00:00:00,25.56%,-2.08%,-6.99%,30.71%,6.39%
2012-12-31 00:00:00,31.40%,17.20%,2.89%,-12.08%,3.62%
2013-12-31 00:00:00,5.42%,33.54%,40.06%,10.00%,13.96%
2014-12-31 00:00:00,37.72%,-9.85%,24.16%,-3.43%,2.20%
2015-12-31 00:00:00,-4.64%,23.27%,19.44%,26.08%,1.75%
2016-12-31 00:00:00,10.03%,1.44%,12.00%,3.03%,-3.49%
2017-12-31 00:00:00,46.11%,-44.78%,37.66%,41.41%,10.66%
2018-12-31 00:00:00,-6.79%,-56.62%,18.74%,3.17%,3.20%


La formula inversa per passare dai rendimenti percentuali a quelli logaritmici è $$r = ln(1 + R)$$

In [34]:
((1 + yearly_returns).apply(np.log)).style.format('{:.2%}')

,AAPL,GE,MSFT,MCD,KO
date,,,,,
2010-12-31 00:00:00,42.57%,18.97%,-8.81%,20.65%,14.31%
2011-12-31 00:00:00,22.76%,-2.10%,-7.24%,26.78%,6.19%
2012-12-31 00:00:00,27.31%,15.87%,2.85%,-12.87%,3.55%
2013-12-31 00:00:00,5.28%,28.92%,33.69%,9.53%,13.07%
2014-12-31 00:00:00,32.01%,-10.37%,21.64%,-3.49%,2.18%
2015-12-31 00:00:00,-4.75%,20.92%,17.76%,23.18%,1.74%
2016-12-31 00:00:00,9.56%,1.43%,11.34%,2.99%,-3.55%
2017-12-31 00:00:00,37.92%,-59.38%,31.96%,34.65%,10.13%
2018-12-31 00:00:00,-7.03%,-83.51%,17.18%,3.12%,3.15%


Proviamo ora a generare un grafico dei rendimenti giornalieri, con due caselle **dropdown** che ci permettano di confrontare due titoli alla volta. 

In [37]:
# Costruiamo due dropdown per selezionare i titoli da confrontare
opts_5 = [c for c in yearly_returns.columns]
dd1_5 = widgets.Dropdown(options=opts_5, description='Titolo 1')
dd2_5 = widgets.Dropdown(options=opts_5, description='Titolo 2', value=STOCKS[1])

# Costruiamo un grafico con i rendimenti giornalieri dei due titoli selezionati
x_sc_5 = bq.DateScale()
y_sc_5 = bq.LinearScale()
lines_5 = bq.Lines(x=prices.pct_change().index, y=[prices.loc[:, c].pct_change() for c in [dd1_5.value, dd2_5.value]], scales={'x': x_sc_5, 'y': y_sc_5},
                   colors=[bq.colorschemes.CATEGORY10[prices.columns.get_loc(c)] for c in [dd1_5.value, dd2_5.value]], display_legend=True,
                   labels=[c for c in [dd1_5.value, dd2_5.value]])
ax_x_5 = bq.Axis(scale=x_sc_5, grid_lines='solid', grid_color='#DDDDDD')
ax_y_5 = bq.Axis(scale=y_sc_5, orientation='vertical', tick_format='0%', grid_lines='solid', grid_color='#DDDDDD')
fig_5 = bq.Figure(marks=[lines_5], axes=[ax_x_5, ax_y_5], title=f'Rendimenti percentuali giornalieri per {dd1_5.value} e {dd2_5.value}', 
                  background_style={'fill': 'white'}, legend_location='top-right', layout={'width': '100%'})

# Definiamo una funzione per aggiornare il grafico, la legenda e il titolo
def update_plot_5(change):
    """ Update the chart with value from the dropdowns."""
    lines_5.y = [prices.loc[:, c].pct_change() for c in [dd1_5.value, dd2_5.value]]
    lines_5.colors = [bq.colorschemes.CATEGORY10[prices.columns.get_loc(c)] for c in [dd1_5.value, dd2_5.value]]
    lines_5.labels = [c for c in [dd1_5.value, dd2_5.value]]
    fig_5.title = f'Rendimenti percentuali giornalieri per {dd1_5.value} e {dd2_5.value}'

# Intercettiamo i valori delle dropdown
dd1_5.observe(update_plot_5, 'value')
dd2_5.observe(update_plot_5, 'value')
    
# Definiamo il layout delle widget
dd_5 = widgets.HBox([dd1_5, dd2_5] , layout={'width': '100%'})
widgets.VBox([dd_5, fig_5])

Dal grafico si vede chiaramente come i rendimenti cambino nel tempo e siano diversi tra i titoli. Questa osservazione introduce un nuovo argomento, la **volatilità**, cioè la variabilità dei rendimenti. Ma questo sarà un punto da trattare in un successivo **notebook**.